In [18]:
%run runaway_functionsv3

In [5]:
workclusters

['ASCC_107',
 'ASCC_114',
 'ASCC_127',
 'ASCC_13',
 'ASCC_16',
 'ASCC_19',
 'ASCC_21',
 'ASCC_32',
 'ASCC_67',
 'ASCC_9',
 'Alessi_20',
 'Alessi_43',
 'Alessi_Teutsch_5',
 'Antalova_2',
 'Archinal_1',
 'Aveni_Hunter_1',
 'BDSB91',
 'BDSB93',
 'BDSB96',
 'BH_121',
 'BH_200',
 'BH_205',
 'BH_217',
 'BH_221',
 'BH_245',
 'BH_54',
 'BH_56',
 'BH_87',
 'BH_92',
 'Barkhatova_1',
 'Basel_18',
 'Basel_8',
 'Berkeley_11',
 'Berkeley_15',
 'Berkeley_47',
 'Berkeley_62',
 'Berkeley_65',
 'Berkeley_7',
 'Berkeley_79',
 'Berkeley_86',
 'Berkeley_87',
 'Berkeley_97',
 'Bica_2',
 'Biurakan_2',
 'Bochum_10',
 'Bochum_11',
 'Bochum_13',
 'COIN-Gaia_16',
 'COIN-Gaia_21',
 'COIN-Gaia_35',
 'COIN-Gaia_41',
 'Collinder_104',
 'Collinder_106',
 'Collinder_107',
 'Collinder_132',
 'Collinder_197',
 'Collinder_205',
 'Collinder_272',
 'Collinder_419',
 'Collinder_421',
 'Collinder_69',
 'Collinder_95',
 'Czernik_1',
 'Czernik_31',
 'Czernik_41',
 'Czernik_6',
 'Czernik_8',
 'Dias_1',
 'Dias_5',
 'Dolidze_16',

In [26]:
dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
dias2021[dias2021['Cluster']=='Basel_8']

Cluster,RA_ICRS,DE_ICRS,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,Dist,e_Dist,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,pc,pc,log(yr),log(yr),,,mag,mag,,,deg,deg
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64
Basel_8,98.5582,8.0381,0.299,31,-0.134,0.066,-2.400,0.140,0.623,0.036,--,--,--,1456,54,7.671,0.236,0.028,0.064,1.083,0.079,clusters1/Basel_8.dat,Cl Basel 8,98.5582,8.0381


In [65]:
cl = Cluster("Basel_8")
np.count_nonzero(~cl.stars_in_region()['RV'].mask)


375

In [4]:
cl = Cluster("Basel_8")
cl.stars_in_region()
t = cl.mymembers
t = cl.stars_in_region()
t = find_cluster(cl.stars_in_region())

np.mean(t['RV']),np.sqrt(np.sum(t['e_RV'])**2/(len(t))**2)

(25.763333333333332, 1.4898182262073862)

In [2]:
find_cluster(cl.stars_in_region())

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str11,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord
339.84392032630,59.00715291679,0.0293,0.0308,0.2402,--,3995-625-1,2008383302396655488,2630.22485000,0.3652,0.0363,-2.873,-2.301,0.038,0.038,3.035,18135.5,3.3302,9.473729,0.937676,9.843719,8.906042,--,--,2393.19263000,2940.37549000,3058125.18769,951.5,1.577e+06,1154,2.171e+06,1124,--,--,0.002776,0.002901,0.003821,0.006722,"339.8439203263,59.00715291679,2630.22485"
339.82384023875,59.01681945374,0.0097,0.0105,1.0890,--,3995-1075-1,2008430237808841600,2657.56860000,0.3583,0.0125,-2.747,-1.911,0.013,0.013,0.986,--,--,9.840384,0.868947,10.178231,9.309284,--,--,2587.60059000,2734.51587000,2181691.83228,1048,1.159e+06,1141,1.498e+06,1215,--,--,0.002804,0.002988,0.003881,0.006869,"339.82384023875,59.01681945374,2657.5686"
339.82166004461,59.00297803329,0.0089,0.0097,0.8784,--,--,2008430203449108224,2525.15747000,0.3648,0.0114,-2.953,-1.724,0.012,0.012,0.988,--,--,13.262117,0.961862,13.648018,12.686156,--,--,2459.89844000,2592.30542000,93346.90312,21.67,4.745e+04,60.24,6.679e+04,44.71,--,--,0.002767,0.003112,0.003849,0.006961,"339.82166004461,59.00297803329,2525.15747"
339.84074866849,59.00514934930,0.0134,0.0139,0.2851,--,--,2008383302404812544,2604.80005000,0.3569,0.0163,-2.626,-1.913,0.018,0.017,1.124,11638.8,3.8049,14.199184,1.058014,14.629308,13.571294,--,--,2466.69116000,2720.19312000,39379.74224,11.71,1.922e+04,51.21,2.956e+04,27.21,--,--,0.002774,0.00402,0.003909,0.007929,"339.84074866849,59.0051493493,2604.80005"
339.85983016055,59.00845000045,0.0134,0.0137,0.3836,--,--,2008384779873564032,2623.42310000,0.3563,0.0168,-2.780,-1.834,0.017,0.016,1.021,10857.5,3.8294,14.490845,1.049156,14.918883,13.869727,--,--,2502.20532000,2719.62866000,30102.94455,6.87,1.472e+04,15.67,2.245e+04,17.55,--,--,0.002766,0.00302,0.003874,0.006894,"339.85983016055,59.00845000045,2623.4231"
339.76936809268,58.98330151380,0.0146,0.0149,2.7984,--,--,2008430066010153216,2691.21167000,0.3452,0.0179,-2.744,-1.877,0.018,0.017,1.106,10748.6,3.8311,14.639523,1.056804,15.072841,14.016037,--,--,2552.40112000,2831.95532000,26250.49507,6.071,1.277e+04,12.93,1.962e+04,16.58,--,--,0.002767,0.002999,0.003889,0.006888,"339.76936809268,58.9833015138,2691.21167"
339.87917790059,59.08169389830,0.0148,0.0148,4.7127,--,--,2008432196314226688,2630.08740000,0.3540,0.0177,-2.698,-1.548,0.019,0.019,0.984,--,--,14.759680,1.113024,15.222409,14.109386,--,--,2508.20801000,2761.05005000,23500.38187,5.307,1.113e+04,15.63,1.8e+04,15.13,--,--,0.002766,0.00318,0.003888,0.007068,"339.87917790059,59.0816938983,2630.0874"
339.89915881624,59.04024321752,0.0151,0.0151,2.6240,--,--,2008431749637657856,2690.27222000,0.3456,0.0189,-2.655,-1.860,0.021,0.020,1.050,9068.6,3.7741,14.804451,1.156634,15.292900,14.136266,--,--,2563.39746000,2837.99658000,22551.01862,5.476,1.043e+04,11.12,1.756e+04,18.51,--,--,0.002768,0.003021,0.003949,0.00697,"339.89915881624,59.04024321752,2690.27222"


In [3]:
cl = Cluster("Basel_8")
print(cl.RV, cl.e_RV)
cl.fast_stars_in_region()


19.2 0.0


RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,e_rmRA,e_rmDE,rRV,e_rRV,v_pec
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,km / s
float64,float64,float64,float64,float64,int32,str10,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord,float64,float64,float32,float32,float64,float32,float64
98.69055275937,8.07569472560,0.0356,0.0302,8.1797,--,733-1687-1,3326026144355977472,1458.14001000,0.6601,0.0421,-1.403,0.651,0.039,0.031,1.529,--,--,9.317637,0.329390,9.419267,9.089877,--,--,1378.07080000,1536.19507000,3530953.30802,1620,2.332e+06,5770,1.833e+06,2216,--,--,0.0028,0.003873,0.004001,0.007874,"98.69055275937,8.0756947256,1458.14001",-1.2133750050067902,2.834125019073486,0.26384768,0.2558477,--,--,21.30802513378318
98.02515215957,8.09905249055,0.0143,0.0142,31.8772,--,733-1181-1,3326201829997700864,1524.07874000,0.6292,0.0183,2.051,-3.571,0.017,0.014,1.133,4553.0,2.0382,10.953504,1.681146,11.758776,10.077630,35.17,0.19,1480.84570000,1572.99548000,782610.13964,172.5,2.703e+05,215.5,7.381e+05,266.9,--,--,0.002766,0.002921,0.0038,0.006721,"98.02515215957,8.09905249055,1524.07874",2.24062499499321,-1.3878749809265138,0.2418477,0.23884769,15.97,0.19,19.040213295718296
98.91793033140,8.40607900769,0.0277,0.0232,30.7213,--,--,3326133660275716480,1192.20020000,0.8168,0.0316,3.212,0.952,0.030,0.024,1.821,--,--,11.122290,1.406081,11.761770,10.355689,50.24,0.77,1148.89038000,1250.99280000,669932.02689,165.4,2.696e+05,163.5,5.713e+05,246.5,--,--,0.002768,0.002867,0.003808,0.006675,"98.9179303314,8.40607900769,1192.2002",3.40162499499321,3.1351250190734863,0.25484768,0.2488477,31.04,0.77,26.141773209726033
98.80943694927,8.02405631161,0.0143,0.0132,14.9497,--,733-1827-1,3325977426545923840,1605.87500000,0.6013,0.0165,2.491,-3.233,0.018,0.016,0.950,--,--,11.189508,1.873466,12.116813,10.243346,54.60,0.20,1569.39270000,1654.23608000,629714.19999,137.1,1.944e+05,153.6,6.336e+05,217.9,--,--,0.002765,0.002919,0.003798,0.006717,"98.80943694927,8.02405631161,1605.875",2.68062499499321,-1.0498749809265138,0.24284768,0.24084769,35.40,0.20,21.913645483039886
98.32807784189,7.85959204576,0.0283,0.0251,17.3699,--,733-2008-1,3325994876995037824,1207.66919000,0.8004,0.0329,-0.988,-7.367,0.032,0.025,1.391,5133.8,2.7982,11.342217,1.351886,11.946281,10.594396,-26.32,1.19,1168.05396000,1258.94336000,547091.22454,125.6,2.275e+05,103.9,4.586e+05,203.2,--,--,0.002767,0.002834,0.00381,0.006644,"98.32807784189,7.85959204576,1207.66919",-0.7983750050067902,-5.183874980926514,0.25684768,0.2498477,-45.52,1.19,30.02419190903883
98.15627576457,8.56722476217,0.0173,0.0162,39.7161,--,--,3326255495610553728,1175.12512000,0.8252,0.0189,-1.074,4.618,0.020,0.017,1.133,4952.4,2.7904,11.348566,1.440937,12.007280,10.566343,40.51,0.21,1149.72412000,1201.94348000,543901.81906,159.3,2.15e+05,135.1,4.706e+05,233.1,--,--,0.002774,0.002872,0.003817,0.00669,"98.15627576457,8.56722476217,1175.12512",-0.8843750050067902,6.801125019073487,0.24484769,0.2418477,21.31,0.21,38.201832951479794
98.22562558636,7.71688625045,0.0154,0.0148,27.6070,--,--,3325989001488975104,1600.60449000,0.5939,0.0186,-2.467,-1.681,0.020,0.015,0.785,--,--,11.375305,1.212525,11.901155,10.688629,13.60,1.29,1553.27344000,1645.29480000,530670.07959,124.9,2.371e+05,116.6,4.204e+05,128.6,--,--,0.002767,0.002841,0.003794,0.006635,"98.22562558636,7.71688625045,1600.60449",-2.2773750050067902,0.5021250190734863,0.24484769,0.23984769,-5.60,1.29,17.693126886092205
97.96487496352,8.3704765

In [7]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster("Berkeley_97")
cir = find_cluster(cl.stars_in_region())
# cir = (cl.fast_stars_in_region())
cir['rmRA'] = cir['pmRA']-cl.pm_ra_cosdec
cir['rmDE'] = cir['pmDE']-cl.pm_dec
cir['e_rmRA'] = cir['e_pmRA']+cl.e_pm_ra_cosdec
cir['e_rmDE'] = cir['e_pmDE']+cl.e_pm_dec
cir['rRV'] = cir['RV']-cl.RV
cir['e_rRV'] = cir['e_RV']+cl.e_RV
cir['Temp. Est'] = 0
cl.plot_traceback_clean(stars=cir)

QSocketNotifier: Can only be used with threads started with QThread
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [8]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster("Berkeley_97")
cld = ClusterDias("Berkeley_97")
isochrone1 = Isochrone(cld)
isochrone2 = Isochrone(cl, Av=3, logage=7)
plot_cmd(cl, isochrones=[isochrone1,isochrone2])
# plot_cmd(cld)



In [48]:
source = 2008384504995669248
object = f"gaia dr3 {source}"
print(object, type(object))
Simbad.query_object("gaia dr3 2008383302396655488")

gaia dr3 2008384504995669248 <class 'str'>


MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
HD 240016,22 39 22.5468,+59 00 25.787,14,14,0.029,0.031,90,A,O,2020yCat.1350....0G,1


In [45]:
cl.runaways()

RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,v_pec,logg,RV,e_RV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
deg,deg,pc,K,K,mas,mas,,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,km / s,log(cm.s**-2),km / s,km / s,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float64,float64,float64,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
339.89581711423,58.99547917199,2620.36230000,9168.0,10920.0,0.0202,0.0197,1.5205,--,--,2008384504995669248,0.3560,0.0241,-0.379,-1.352,0.025,0.024,1.086,15.384143,1.182070,15.883185,14.701116,2461.53345000,2759.61401000,0.00277,0.003852,0.004229,0.008081,"339.89581711423,58.99547917199,2620.3623",2.3543571910858154,0.5321785192489623,29.98008203419036,3.7061,--,--,13221.76503,3.482,6055,14.81,1.044e+04,18.25,--,--


In [44]:
for runaway in cl.runaways():
    source = runaway['Source']
    print(source)
    
    t = Simbad.query_object(f"gaia dr3 {source}")['MAIN_ID']
    print(t)

2008384504995669248


/usr/lib/python3/dist-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR3 2008384504995669248
  warnings.warn("Warning: The script line number %i raised "


TypeError: 'NoneType' object is not subscriptable

In [41]:
from astroquery.simbad import Simbad
t = Simbad.query_object("gaia dr3 2008384504995669248")
t

/usr/lib/python3/dist-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR3 2008384504995669248
  warnings.warn("Warning: The script line number %i raised "


In [13]:
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
workclusters = []
for clustername in dias2021[maskplx & maskage][:]['Cluster']:
    if clustername not in ['ASCC_79','BH_164','BH_23','Collinder_135','Collinder_140','Gulliver_9','IC_2391','IC_2602','Mamajek_1','Platais_8','UPK_535','UPK_606','UPK_640','Berkeley_59','COIN-Gaia_37','Ivanov_4','LP_1937','Sigma_Ori','UBC_632']:
        workclusters.append(clustername)
        # print(clustername)
        # cl = Cluster(clustername)
        # cl.prepare_trace()
# cl.stars_in_region()
# cl.stars_in_region()

In [ ]:
workclusters_mask = [cluster in workclusters for cluster in dias2021['Cluster']]
workclusterstable = dias2021[workclusters_mask]

rvexists = workclusterstable['RV'].mask
nrvgood = workclusterstable['NRV'] > 4

workclusterstable[~rvexists & nrvgood]

In [ ]:
def runaways_from(clustername):
    cl = Cluster(clustername)
    fs4giesler = cl.fast_stars_in_region()
    outputs = os.listdir(f"/home/surodeep/suro_aiu/traceback/cluster_runaway/{cl.name}/runaways/")
    linenos = []
    for output in outputs:
        #print(output)
        if 'run' in output:
            linenos.append(int(output.split("+")[1].replace(".out","")))
    linenos.sort()
    # print(linenos)
    i=np.array(linenos)-3
    def source_of(lineno, input_table):
        return input_table[lineno-2]['Source']
    return fs4giesler[i]
for cluster in workclusters:
    t = runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Teff']
    
    if t>10000:
        print(cluster, t,runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Source'])